In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema('microns_ta3p100')

Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.ProofreadLabelOrphan() & "segment_id=648518346341352006"

In [4]:
#ta3p100.CleansedMeshOrphan()

In [5]:
@schema
class CoarseLabelOrphan(dj.Computed):
    definition = """
    # Vertex labels for ta3p100.ProofreadLabel did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> ta3p100.ProofreadLabelOrphan
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as ta3p100.ProofreadLabelOrphan
    """
    
    key_source = ta3p100.ProofreadLabelOrphan & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (ta3p100.ProofreadLabelOrphan & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (ta3p100.CleansedMeshOrphan & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [6]:
start = time.time()
CoarseLabelOrphan.populate(reserve_jobs=True)
print(time.time() - start)

648518346341379688
Segment 648518346341379688 vertex labels regenerated in: 1.1716926097869873 seconds.
648518346341380192
Segment 648518346341380192 vertex labels regenerated in: 0.9989771842956543 seconds.
648518346341380839
Segment 648518346341380839 vertex labels regenerated in: 0.8861536979675293 seconds.
648518346341381654
Segment 648518346341381654 vertex labels regenerated in: 0.9030869007110596 seconds.
648518346341383270
Segment 648518346341383270 vertex labels regenerated in: 1.1306793689727783 seconds.
648518346341384074
Segment 648518346341384074 vertex labels regenerated in: 1.7119114398956299 seconds.
648518346341384355
Segment 648518346341384355 vertex labels regenerated in: 0.9805459976196289 seconds.
648518346341386292
Segment 648518346341386292 vertex labels regenerated in: 1.9196445941925049 seconds.
648518346341387646
Segment 648518346341387646 vertex labels regenerated in: 0.9499354362487793 seconds.
648518346341388366
Segment 648518346341388366 vertex labels rege

In [7]:
#(schema.jobs & "table_name='__coarse_label_orphan'")#.delete()

In [8]:
#ta3p100.CoarseLabelOrphan()